# 일대일 병합(수평 결합)
- 수평 결합 : 데이터테이블의 열을 다른 테이블 열과 병합하는 것
- 이 장에서는 양쪽 파일 모두에서 병합 기준열의 중복은 없음
---
### 데이터는 NLS 2종 사용
- 한 행에 한 사람의 데이터 
- 취업/학력/소득 정보 데이터 & 응답자의 부모소득 및 학력 데이터

In [1]:
import pandas as pd
nls97 = pd.read_csv('data/nls97f.csv')
nls97.set_index('personid', inplace = True)
nls97add = pd.read_csv('data/nls97add.csv')

In [2]:
# 데이터 살펴보기
nls97.head()

,gender,birthmonth,birthyear,highestgradecompleted,maritalstatus,childathome,childnotathome,wageincome,weeklyhrscomputer,weeklyhrstv,...,colenroct13,colenrfeb14,colenroct14,colenrfeb15,colenroct15,colenrfeb16,colenroct16,colenrfeb17,colenroct17,originalid
personid,,,,,,,,,,,,,,,,,,,,,
100061,Female,5,1980,13.0,Married,4.0,0.0,12500.0,10 hours or more a week,11 to 20 hours a week,...,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,8245
100139,Male,9,1983,12.0,Married,2.0,0.0,120000.0,1 to 3 hours a week,3 to 10 hours a week,...,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,3962
100284,Male,11,1984,7.0,Never-married,1.0,0.0,58000.0,None,11 to 20 hours a week,...,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,3571
100292,Male,4,1982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,NaN,NaN,NaN,NaN,2979
100583,Male,1,1980,13.0,Married,4.0,0.0,30000.0,Less than 1 hour a week,3 to 10 hours a week,...,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,8511


In [3]:
nls97.shape

(8984, 89)

In [4]:
nls97add.head()

,originalid,motherage,parentincome,fatherhighgrade,motherhighgrade
0,1,26,-3,16,8
1,2,19,-4,17,15
2,3,26,63000,-3,12
3,4,33,11700,12,12
4,5,34,-3,12,12


### Originalid의 유일값 개수가 행 수와 같은지 확인

In [6]:
nls97.originalid.nunique() == nls97.shape[0]

True

In [7]:
nls97add.originalid.nunique() == nls97add.shape[0]

True

### 불일치하는 ID 생성
- 실습을 위해 데이터를 고의로 훼손

In [8]:
nls97 = nls97.sort_values('originalid')

In [9]:
nls97add = nls97add.sort_values('originalid')

In [10]:
nls97.iloc[0:2, -1] = nls97[0:2].originalid + 10000

In [11]:
nls97.originalid.head(2)

personid
135335    10001
999406    10002
Name: originalid, dtype: int64

In [14]:
nls97add.iloc[0:2, 0] = nls97add[0:2].originalid + 20000

In [15]:
nls97add.originalid.head(2)

0    20001
1    20002
Name: originalid, dtype: int64

### join으로 왼쪽 조인 수행
- 병합 기준 열이 인덱스인 경우, join 메서드를 사용하는 것이 왼쪽 조인을 수행하는 가장 쉬운 방법

In [16]:
nlsnew = nls97.join(nls97add.set_index(['originalid']))

In [17]:
nlsnew.loc[nlsnew.originalid > 9999, ['originalid', 'gender', 'birthyear', 'motherage', 'parentincome']]

,originalid,gender,birthyear,motherage,parentincome
personid,,,,,
135335,10001,Female,1981,NaN,NaN
999406,10002,Male,1982,NaN,NaN


### merge로 왼쪽 조인 수행

In [18]:
nlsnew = pd.merge(nls97, nls97add, on=['originalid'], how='left')

In [19]:
nlsnew.loc[nlsnew.originalid > 9999, ['originalid', 'gender', 'birthyear', 'motherage', 'parentincome']]

,originalid,gender,birthyear,motherage,parentincome
0,10001,Female,1981,NaN,NaN
1,10002,Male,1982,NaN,NaN


### 오른쪽 조인 수행

In [20]:
nlsnew = pd.merge(nls97, nls97add, on=['originalid'], how='right')
nlsnew.loc[nlsnew.originalid > 9999, ['originalid', 'gender', 'birthyear', 'motherage', 'parentincome']]

,originalid,gender,birthyear,motherage,parentincome
0,20001,NaN,NaN,26,-3
1,20002,NaN,NaN,19,-4


### 내부 조인 수행

In [21]:
nlsnew = pd.merge(nls97, nls97add, on=['originalid'], how='inner')
nlsnew.loc[nlsnew.originalid > 9999, ['originalid', 'gender', 'birthyear', 'motherage', 'parentincome']]

,originalid,gender,birthyear,motherage,parentincome


### 외부 조인 수행

In [22]:
nlsnew = pd.merge(nls97, nls97add, on=['originalid'], how='outer')
nlsnew.loc[nlsnew.originalid > 9999, ['originalid', 'gender', 'birthyear', 'motherage', 'parentincome']]

,originalid,gender,birthyear,motherage,parentincome
0,10001,Female,1981.0,NaN,NaN
1,10002,Male,1982.0,NaN,NaN
8984,20001,NaN,NaN,26.0,-3.0
8985,20002,NaN,NaN,19.0,-4.0


### 📍ID의 불일치를 검사하는 함수 작성
- 왼쪽 데이터프레임, 오른쪽 데이터프레임, 병합 기준 열을 매개변수로 받음
- 외부조인을 수행하여, 병합 기준 값이 한쪽 또는 양쪽 데이터프레임에 모두 있는지 확인하고자 하기 때문

In [23]:
def checkmerge(dfleft, dfright, idvar):
    dfleft['inleft'] = 'Y'
    dfright['inright'] = 'Y'
    dfboth = pd.merge(dfleft[[idvar, 'inleft']], dfright[[idvar, 'inright']], on=[idvar], how='outer')
    dfboth.fillna('N', inplace = True)
    print(pd.crosstab(dfboth.inleft, dfboth.inright))

In [24]:
checkmerge(nls97, nls97add, 'originalid')

inright  N     Y
inleft          
N        0     2
Y        2  8982
